In [ ]:
# Importing essential libraries

# This module is not included in Colab Notebook by default and needs to be installed seperately.
!pip install tensorflow-decision-forests==1.3.0
import pandas as pd
from sklearn.metrics import mean_squared_error

train_data = pd.read_csv('https://pastebin.com/raw/Wz1CNwBv')
test_data = pd.read_csv('https://pastebin.com/raw/6wZnGdRH')

y = train_data.SalePrice

In [ ]:
train_data['SalePrice'].fillna(train_data.SalePrice.mean(), inplace = True)
x_test_data = test_data.drop ('Id', axis = 1)
train_data.drop ('Id', axis = 1, inplace = True)

In [ ]:
x = train_data
x_test = x_test_data

In [ ]:
# Normalising data in all columns to lie between 0 and 1

# normalized_x = (x-x.min())/(x.max()-x.min())
# normalized_x_test = (x_test-x_test.min())/(x_test.max()-x_test.min())

# Normalizing data further increased the RMSE, hence commented out

In [ ]:
# Applying Random Forest Regression to the data using SciKit Learn

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_percentage_error

# model_RFR = RandomForestRegressor(n_estimators=1000)
# model_RFR.fit(x, y)
# y_pred = model_RFR.predict(x_test)

In [ ]:
# Applying Random Forest using TensorFlow

import tensorflow as tf
import tensorflow_decision_forests as tfdf
print("TensorFlow v" + tf.__version__ + tfdf.__version__)

label = 'SalePrice'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(x, label=label, task = tfdf.keras.Task.REGRESSION)


rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, num_trees = 400)
rf.fit(x=train_ds)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    x_test,
    task = tfdf.keras.Task.REGRESSION)

preds = rf.predict(test_ds)
output_rf = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': preds.squeeze()})

output_rf.head()

TensorFlow v2.12.01.3.0
Use /tmp/tmpbx9sfyp1 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.993174. Found 1460 examples.
Training model...
Model trained in 0:00:09.218492
Compiling model...
Model compiled.
2/2 [==============================] - 2s 61ms/step


,Id,SalePrice
0,1461,126499.890625
1,1462,154204.093750
2,1463,179424.859375
3,1464,184213.140625
4,1465,193684.171875


In [ ]:
# Applying Gradient Boosted Trees Model using TensorFlow

import tensorflow as tf
import tensorflow_decision_forests as tfdf

label = 'SalePrice'
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(x, label=label, task = tfdf.keras.Task.REGRESSION)

gbt = tfdf.keras.GradientBoostedTreesModel(task = tfdf.keras.Task.REGRESSION)
gbt.fit(x=train_ds)

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    x_test,
    task = tfdf.keras.Task.REGRESSION)

preds = gbt.predict(test_ds)
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': preds.squeeze()})

output.head()

Use /tmp/tmpd_kl6jfs as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.266311. Found 1460 examples.
Training model...
Model trained in 0:00:06.169829
Compiling model...
Model compiled.
2/2 [==============================] - 3s 70ms/step


,Id,SalePrice
0,1461,125297.554688
1,1462,153793.968750
2,1463,191341.781250
3,1464,188541.640625
4,1465,195458.296875


In [ ]:
# Taking mean of the two outputs

output.SalePrice = (output.SalePrice + output_rf.SalePrice)/2
output

,Id,SalePrice
0,1461,125898.718750
1,1462,153999.031250
2,1463,185383.312500
3,1464,186377.390625
4,1465,194571.234375
...,...,...
1454,2915,83544.093750
1455,2916,80820.750000
1456,2917,167870.031250
1457,2918,120562.156250


In [ ]:
# Applying Cart Model using TensorFlow

# import tensorflow as tf
# import tensorflow_decision_forests as tfdf

# label = 'SalePrice'
# train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(x, label=label, task = tfdf.keras.Task.REGRESSION)

# cart = tfdf.keras.CartModel(task = tfdf.keras.Task.REGRESSION)
# cart.fit(x=train_ds)

# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
#     x_test,
#     task = tfdf.keras.Task.REGRESSION)

# preds = cart.predict(test_ds)
# output = pd.DataFrame({'Id': test_data.Id,
#                        'SalePrice': preds.squeeze()})

# output.head()

In [ ]:
# Applying Linear Regression to the data

# from sklearn import linear_model
# regr = linear_model.LinearRegression()
# regr.fit(x, y)
# y_pred = regr.predict(x_test)

In [ ]:
# output = pd.DataFrame({'Id': test_data.Id, 'SalePrice':y_pred})
output.to_csv('submission_nimit.csv', index = False)

In [ ]:
from google.colab import files
files.download('submission_nimit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

NOTES

*   I noticed that while applying Random Forest Regression once through Scikit Learn and once through TensorFlow module with default configuration, the score is significantly different - 0.195 with scikit and 0.1428 with tensorflow. I couldn't figure out why this was so; I decided to use tensorflow in my final submission.
*   Only the model which yielded the lowest RMSE is being executed. Rest all are commented out which I tried. However, I have included the csv files to these approaches also as my top 5 submissions on Kaggle Platform.


-- Submitted by Nimit Khandelwal
